In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0_pcs7', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_pcs7...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [3]:
wau = mysql.read_table(spark, "pubg_gi", "all_traffic_activity_weekly")

## PC

In [4]:
wau.createOrReplaceTempView("wau")
sql_str = """
    select startdate, enddate, sum(wau) as wau
    from wau
    where startdate in ("2022-08-12", "2022-08-19", "2022-08-26", "2022-09-02") and device = "PC"
    group by 1, 2
"""

wau_4wk = spark.sql(sql_str)

In [6]:
wau_4wk.orderBy("startdate").toPandas()

,startdate,enddate,wau
0,2022-08-12,2022-08-18,4565299
1,2022-08-19,2022-08-25,4501558
2,2022-08-26,2022-09-01,4532848
3,2022-09-02,2022-09-08,4586889


In [7]:
wau.createOrReplaceTempView("wau")
sql_str = """
    select startdate, enddate, sum(wau) as wau
    from wau
    where startdate in ("2022-03-16", "2022-03-23", "2022-03-30", "2022-04-06") and device = "PC"
    group by 1, 2
"""

pcs6_wau_4wk = spark.sql(sql_str)

In [8]:
pcs6_wau_4wk.orderBy("startdate").toPandas()

,startdate,enddate,wau
0,2022-03-16,2022-03-22,4954557
1,2022-03-23,2022-03-29,6226397
2,2022-03-30,2022-04-05,6309332
3,2022-04-06,2022-04-12,5009617


In [11]:
pcs6_gcoin = load_data_mart('pc', "2022-03-16", "2022-04-12", "gcoin_use").where(col('event_name').like("%pcs%"))
pcs6_gcoin = pcs6_gcoin.withColumn("start_date", when((col("date") >= "2022-03-16") & (col('date') <= "2022-03-22"), "2022-03-16"). \
                                  when((col("date") >= "2022-03-23") & (col('date') <= "2022-03-29"), "2022-03-23"). \
                                  when((col("date") >= "2022-03-30") & (col('date') <= "2022-04-05"), "2022-03-30"). \
                                  when((col("date") >= "2022-04-06") & (col('date') <= "2022-04-12"), "2022-04-06"). \
                                  otherwise(lit(None))
                                )

In [12]:
pcs6_gcoin.groupBy("start_date").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).orderBy("start_date").toPandas()

,start_date,paid_use,total_use
0,2022-03-16,74265946,109324730
1,2022-03-23,12785650,28708730
2,2022-03-30,10149040,19802330
3,2022-04-06,12307830,21510970


In [13]:
before_workshop_wsus_1 = load_data_mart('pc', '2022-05-11', "2022-06-07", "gcoin_use").where(col("event_name").like("%wsus%"))
before_workshop_wsus_2 = load_data_mart('pc', '2022-06-08', "2022-07-05", "gcoin_use").where(col("event_name").like("%wsus%"))

after_workshop_wsus_1 = load_data_mart('pc', '2022-07-13', "2022-08-09", "gcoin_use").where(col("event_name").like("%wsus%"))
after_workshop_wsus_2 = load_data_mart('pc', '2022-08-10', "2022-09-06", "gcoin_use").where(col("event_name").like("%wsus%"))

In [14]:
before_workshop_wsus_1.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,702185151,1096487600


In [15]:
before_workshop_wsus_2.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,1468762475,1980513400


In [16]:
after_workshop_wsus_1.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,694732370,994252600


In [17]:
after_workshop_wsus_2.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,1478755945,1846405000


## CONSOLE

In [18]:
wau.createOrReplaceTempView("wau")
sql_str = """
    select startdate, enddate, sum(wau) as wau
    from wau
    where startdate in ("2022-08-19", "2022-08-26", "2022-09-02") and device = "CONSOLE"
    group by 1, 2
"""

wau_3wk = spark.sql(sql_str)
wau_3wk.orderBy("startdate").toPandas()

,startdate,enddate,wau
0,2022-08-19,2022-08-25,670794
1,2022-08-26,2022-09-01,672968
2,2022-09-02,2022-09-08,676958


In [19]:
wau.createOrReplaceTempView("wau")
sql_str = """
    select startdate, enddate, sum(wau) as wau
    from wau
    where startdate in ("2022-03-24", "2022-03-31", "2022-04-07") and device = "CONSOLE"
    group by 1, 2
"""

pcs6_wau_3wk = spark.sql(sql_str)
pcs6_wau_3wk.orderBy("startdate").toPandas()

,startdate,enddate,wau
0,2022-03-24,2022-03-30,725847
1,2022-03-31,2022-04-06,734698
2,2022-04-07,2022-04-13,727668


In [20]:
pcs6_gcoin = load_data_mart('console', "2022-03-24", "2022-04-13", "gcoin_use").where(col('event_name').like("%pcs%"))
pcs6_gcoin = pcs6_gcoin.withColumn("start_date", when((col("date") >= "2022-03-24") & (col('date') <= "2022-03-30"), "2022-03-24"). \
                                  when((col("date") >= "2022-03-31") & (col('date') <= "2022-04-06"), "2022-03-31"). \
                                  when((col("date") >= "2022-04-07") & (col('date') <= "2022-04-13"), "2022-04-07"). \
                                  otherwise(lit(None))
                                )
pcs6_gcoin.groupBy("start_date").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).orderBy("start_date").toPandas()

,start_date,paid_use,total_use
0,2022-03-24,9457390,13970900
1,2022-03-31,2411960,4341140
2,2022-04-07,1589040,3049490


In [21]:
before_workshop_wsus_1 = load_data_mart('console', '2022-05-19', "2022-06-08", "gcoin_use").where(col("event_name").like("%wsus%"))
before_workshop_wsus_2 = load_data_mart('console', '2022-06-16', "2022-07-06", "gcoin_use").where(col("event_name").like("%wsus%"))

after_workshop_wsus_1 = load_data_mart('console', '2022-07-21', "2022-08-10", "gcoin_use").where(col("event_name").like("%wsus%"))
after_workshop_wsus_2 = load_data_mart('console', '2022-08-18', "2022-09-07", "gcoin_use").where(col("event_name").like("%wsus%"))

In [22]:
before_workshop_wsus_1.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,30479970,51677200


In [23]:
before_workshop_wsus_2.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,42761750,68067200


In [24]:
after_workshop_wsus_1.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,34953230,58502800


In [25]:
after_workshop_wsus_2.select(sum('paid_use').alias("paid_use"), sum(col("free_use") + col("paid_use")).alias('total_use')).toPandas()

,paid_use,total_use
0,49091270,72530800
